# Maternal Health Risk Prediction with Explainability
This notebook uses a Random Forest classifier and integrates SHAP to explain why a prediction is made.

In [2]:
# Install SHAP if needed
!pip install shap

Defaulting to user installation because normal site-packages is not writeable
  Using cached shap-0.47.2-cp312-cp312-win_amd64.whl.metadata (25 kB)
Using cached shap-0.47.2-cp312-cp312-win_amd64.whl (545 kB)



[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import shap
import matplotlib.pyplot as plt

C:\Users\Abhin M\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load data (replace this with your actual dataset)
df = pd.read_csv('/kaggle/input/maternal-health-risk-data/Maternal Health Risk Data Set.csv')
X = df.drop(columns='RiskLevel')
y = df['RiskLevel']
X = pd.get_dummies(X)
y = y.map({'low risk': 0, 'mid risk': 1, 'high risk': 2})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
print(classification_report(y_test, rf_model.predict(X_test)))

In [ ]:
# SHAP explainability
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test)

In [ ]:
# SHAP summary plot
shap.summary_plot(shap_values, X_test)

In [ ]:
# Force plot for a single prediction
shap.initjs()
i = 0  # Change this to inspect other samples
shap.force_plot(explainer.expected_value[2], shap_values[2][i], X_test.iloc[i])

In [ ]:
# User-friendly text explanation for top 3 features
instance = X_test.iloc[i]
shap_vals = shap_values[2][i]
top_indices = np.argsort(np.abs(shap_vals))[-3:]
print(f"\n🩺 Prediction Explanation for Sample {i}:")
for idx in reversed(top_indices):
    feature_name = X_test.columns[idx]
    feature_value = instance[feature_name]
    contribution = shap_vals[idx]
    direction = "increased" if contribution > 0 else "decreased"
    print(f" - {feature_name} = {feature_value} → {direction} the risk")